In [310]:
# this will eventually be taken out of a ipython notebook and into a .py file, so don't worry about making it pretty. 
# any data exploration done here will be transferred to the data exploration file.

In [311]:
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBClassifier

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.precision', 3)

df = pd.read_csv('/Users/eloiserosen/Downloads/train.csv')
df_test = pd.read_csv('/Users/eloiserosen/Downloads/test.csv')

#df = pd.read_csv('/Users/eloiserosen/Downloads/train_small_10k.csv')
#df_test = pd.read_csv('/Users/eloiserosen/Downloads/test_small_10k.csv')
#del df['Unnamed: 0']
#del df_test['Unnamed: 0']

In [312]:
df.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2003-02-19 18:00:00,ASSAULT,BATTERY,Wednesday,RICHMOND,NONE,500 Block of JOHN F KENNEDY DR,-122.466,37.773
1,2003-02-19 18:00:00,LARCENY/THEFT,ATTEMPTED THEFT FROM LOCKED VEHICLE,Wednesday,NORTHERN,NONE,300 Block of GROVE ST,-122.422,37.778
2,2003-02-19 18:00:00,OTHER OFFENSES,MISCELLANEOUS STATE MISDEMEANOR,Wednesday,NORTHERN,"ARREST, BOOKED",POST ST / LARKIN ST,-122.418,37.787
3,2003-02-19 18:00:00,ASSAULT,ATTEMPTED HOMICIDE WITH A GUN,Wednesday,NORTHERN,JUVENILE BOOKED,1500 Block of FILLMORE ST,-122.433,37.784
4,2003-02-19 18:00:00,LARCENY/THEFT,PETTY THEFT FROM LOCKED AUTO,Wednesday,TARAVAL,NONE,800 Block of PACHECO ST,-122.470,37.754


In [313]:
df_test.head()

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,874262,2003-02-14 17:30:00,Friday,CENTRAL,300 Block of STOCKTON ST,-122.407,37.789
1,874263,2003-02-14 17:28:00,Friday,NORTHERN,HAYES ST / LAGUNA ST,-122.426,37.776
2,874264,2003-02-14 17:20:00,Friday,INGLESIDE,200 Block of CHENERY ST,-122.427,37.738
3,874265,2003-02-14 17:20:00,Friday,PARK,400 Block of ASHBURY ST,-122.447,37.771
4,874266,2003-02-14 17:18:00,Friday,MISSION,1000 Block of GUERRERO ST,-122.423,37.754


In [314]:
# code to generate small versions of files

'''
df_small = df.tail(10000)
df_small.to_csv('train_small_10k.csv')

df_test_small = df_test.tail(10000)
df_test_small.to_csv('test_small_10k.csv')
'''


"\ndf_small = df.tail(10000)\ndf_small.to_csv('train_small_10k.csv')\n\ndf_test_small = df_test.tail(10000)\ndf_test_small.to_csv('test_small_10k.csv')\n"

In [315]:
def clean_data(df):
    feature_list=df.columns.tolist()
    
    # drop columns we don't need
    if 'Descript' in feature_list:
        del df['Descript']
    if 'Resolution' in feature_list:
        del df['Resolution']
    
    # create columns based on timestamp
    date_time = pd.to_datetime(df['Dates'])
    year = date_time.dt.year
    df['Year'] = year
    month = date_time.dt.month
    df['Month'] = month
    week = date_time.dt.week
    df['Week'] = week
    day = date_time.dt.day
    df['Day'] = day
    hour = date_time.dt.hour
    df['Hour'] = hour
    minute = date_time.dt.minute
    time = hour*60+minute # counting minutes
    df['Time'] = time
    del df['Dates']
    
    # column to indicate if address was on a block
    df['Block'] = df['Address'].str.contains('Block').map(int)
    
    # drop remaining address info for now
    del df['Address']
    
    return df
    

    

In [316]:
df = clean_data(df)
df.head()


,Category,DayOfWeek,PdDistrict,X,Y,Year,Month,Week,Day,Hour,Time,Block
0,ASSAULT,Wednesday,RICHMOND,-122.466,37.773,2003,2,8,19,18,1080,1
1,LARCENY/THEFT,Wednesday,NORTHERN,-122.422,37.778,2003,2,8,19,18,1080,1
2,OTHER OFFENSES,Wednesday,NORTHERN,-122.418,37.787,2003,2,8,19,18,1080,0
3,ASSAULT,Wednesday,NORTHERN,-122.433,37.784,2003,2,8,19,18,1080,1
4,LARCENY/THEFT,Wednesday,TARAVAL,-122.470,37.754,2003,2,8,19,18,1080,1


In [317]:
number_categories = df.Category.nunique()
print number_categories

35


In [318]:
df_test = clean_data(df_test)
df_test.head()

,Id,DayOfWeek,PdDistrict,X,Y,Year,Month,Week,Day,Hour,Time,Block
0,874262,Friday,CENTRAL,-122.407,37.789,2003,2,7,14,17,1050,1
1,874263,Friday,NORTHERN,-122.426,37.776,2003,2,7,14,17,1048,0
2,874264,Friday,INGLESIDE,-122.427,37.738,2003,2,7,14,17,1040,1
3,874265,Friday,PARK,-122.447,37.771,2003,2,7,14,17,1040,1
4,874266,Friday,MISSION,-122.423,37.754,2003,2,7,14,17,1038,1


In [319]:
# not sure why this is being such a cunt when i try to put it in the clean_data def

# set up dummies
dummy_DayOfWeek = pd.get_dummies(df['DayOfWeek'], prefix='Day')
del dummy_DayOfWeek['Day_Friday']
del df['DayOfWeek']
df = df.join(dummy_DayOfWeek)
dummy_PdDistrict = pd.get_dummies(df['PdDistrict'], prefix='District')
del dummy_PdDistrict['District_SOUTHERN']
del df['PdDistrict']
df = df.join(dummy_PdDistrict)


dummy_DayOfWeek = pd.get_dummies(df_test['DayOfWeek'], prefix='Day')
del dummy_DayOfWeek['Day_Friday']
del df_test['DayOfWeek']
df_test = df_test.join(dummy_DayOfWeek)
dummy_PdDistrict = pd.get_dummies(df_test['PdDistrict'], prefix='District')
del dummy_PdDistrict['District_SOUTHERN']
del df_test['PdDistrict']
df_test = df_test.join(dummy_PdDistrict)

In [320]:
df.head()

,Category,X,Y,Year,Month,Week,Day,Hour,Time,Block,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday,District_BAYVIEW,District_CENTRAL,District_INGLESIDE,District_MISSION,District_NORTHERN,District_PARK,District_RICHMOND,District_TARAVAL,District_TENDERLOIN
0,ASSAULT,-122.466,37.773,2003,2,8,19,18,1080,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
1,LARCENY/THEFT,-122.422,37.778,2003,2,8,19,18,1080,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
2,OTHER OFFENSES,-122.418,37.787,2003,2,8,19,18,1080,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
3,ASSAULT,-122.433,37.784,2003,2,8,19,18,1080,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
4,LARCENY/THEFT,-122.470,37.754,2003,2,8,19,18,1080,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0


In [321]:
df_test.head()

,Id,X,Y,Year,Month,Week,Day,Hour,Time,Block,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday,District_BAYVIEW,District_CENTRAL,District_INGLESIDE,District_MISSION,District_NORTHERN,District_PARK,District_RICHMOND,District_TARAVAL,District_TENDERLOIN
0,874262,-122.407,37.789,2003,2,7,14,17,1050,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,874263,-122.426,37.776,2003,2,7,14,17,1048,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,874264,-122.427,37.738,2003,2,7,14,17,1040,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,874265,-122.447,37.771,2003,2,7,14,17,1040,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,874266,-122.423,37.754,2003,2,7,14,17,1038,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


## Target Vector and Feature Matrix

In [322]:
#target vector y
y = df['Category']
y.head()

0           ASSAULT
1     LARCENY/THEFT
2    OTHER OFFENSES
3           ASSAULT
4     LARCENY/THEFT
Name: Category, dtype: object

In [323]:
#Matrix of X's.
X = df
del X['Category']
X.head()

,X,Y,Year,Month,Week,Day,Hour,Time,Block,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday,District_BAYVIEW,District_CENTRAL,District_INGLESIDE,District_MISSION,District_NORTHERN,District_PARK,District_RICHMOND,District_TARAVAL,District_TENDERLOIN
0,-122.466,37.773,2003,2,8,19,18,1080,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
1,-122.422,37.778,2003,2,8,19,18,1080,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
2,-122.418,37.787,2003,2,8,19,18,1080,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
3,-122.433,37.784,2003,2,8,19,18,1080,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
4,-122.470,37.754,2003,2,8,19,18,1080,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0


In [324]:
# delete the id column for now so that we can run our classifier
kaggle_X = df_test
ids = kaggle_X['Id']
del kaggle_X['Id']


xgb = XGBClassifier(objective = 'multi:softprob', max_depth = 6, learning_rate = 1.0, max_delta_step = 1, seed=0)
xgb.fit(X, y)
predictions = pd.DataFrame(xgb.predict_proba(kaggle_X), columns=xgb.classes_)

# grid search below
'''
xgb = XGBClassifier()

from sklearn.grid_search import GridSearchCV
param_grid = {'max_depth': np.arange(3, 12)}
grid = GridSearchCV(xgb, param_grid, n_jobs=4)
grid.fit(X, y)
print grid.grid_scores_
print grid.best_score_
print grid.best_estimator_
print grid.best_params_
'''

In [325]:
predictions.head()

,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,FAMILY OFFENSES,FORGERY/COUNTERFEITING,FRAUD,GAMBLING,KIDNAPPING,LARCENY/THEFT,LIQUOR LAWS,LOITERING,MISSING PERSON,NON-CRIMINAL,OTHER OFFENSES,PROSTITUTION,ROBBERY,RUNAWAY,SECONDARY CODES,SEX OFFENSES FORCIBLE,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,1.642e-05,0.117,4.889e-05,3.717e-05,8.380e-02,6.964e-05,2.858e-06,4.045e-03,1.085e-04,2.621e-04,3.319e-06,8.434e-04,4.455e-03,2.501e-04,6.579e-06,0.423,9.160e-06,1.159e-05,1.661e-03,0.020,0.053,6.654e-05,9.059e-03,3.507e-05,6.505e-04,5.178e-04,2.854e-05,1.915e-04,8.393e-06,1.625e-01,3.908e-03,0.059,0.013,0.043,1.313e-04
1,9.663e-05,0.005,9.355e-07,3.751e-05,4.077e-06,2.389e-07,1.132e-05,2.261e-02,1.984e-04,2.259e-05,3.194e-06,3.086e-06,3.190e-04,2.054e-05,9.797e-07,0.277,3.194e-06,2.808e-06,1.351e-04,0.035,0.575,8.573e-06,2.107e-03,1.433e-04,9.558e-05,3.780e-05,1.001e-05,3.087e-06,1.985e-06,1.294e-04,8.325e-06,0.008,0.061,0.014,8.000e-05
2,6.736e-05,0.198,6.141e-06,2.124e-04,7.005e-02,6.549e-06,2.514e-06,1.432e-02,6.515e-04,2.534e-04,1.611e-04,7.956e-03,1.307e-04,5.707e-05,1.641e-05,0.019,7.576e-05,2.707e-06,9.518e-03,0.023,0.145,1.215e-06,1.256e-03,7.252e-04,3.161e-04,3.606e-05,3.877e-05,1.102e-04,5.919e-05,3.665e-03,2.458e-04,0.029,0.357,0.118,1.979e-04
3,6.967e-04,0.029,3.461e-06,1.309e-05,1.864e-02,1.241e-05,2.305e-06,1.213e-02,6.565e-03,1.591e-05,1.097e-05,1.423e-03,9.784e-05,1.969e-05,3.781e-06,0.122,8.539e-06,1.352e-06,1.326e-04,0.006,0.700,3.483e-05,8.458e-05,3.220e-04,2.706e-05,5.642e-04,9.595e-06,2.767e-06,2.247e-06,5.337e-02,3.054e-06,0.017,0.003,0.028,1.382e-04
4,2.122e-04,0.151,4.230e-06,6.719e-05,2.408e-02,8.666e-06,1.868e-06,5.115e-04,8.937e-04,3.886e-05,1.506e-05,3.084e-03,5.830e-04,3.931e-05,3.847e-06,0.105,6.509e-04,1.933e-06,4.642e-02,0.130,0.334,1.852e-06,1.590e-03,4.888e-05,5.988e-05,1.896e-03,1.915e-05,9.977e-05,4.747e-05,3.280e-03,7.794e-05,0.065,0.117,0.013,2.555e-04


In [326]:
# put the id column back
predictions = pd.concat([ids, predictions], axis=1)
predictions.head()

,Id,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,FAMILY OFFENSES,FORGERY/COUNTERFEITING,FRAUD,GAMBLING,KIDNAPPING,LARCENY/THEFT,LIQUOR LAWS,LOITERING,MISSING PERSON,NON-CRIMINAL,OTHER OFFENSES,PROSTITUTION,ROBBERY,RUNAWAY,SECONDARY CODES,SEX OFFENSES FORCIBLE,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,874262,1.642e-05,0.117,4.889e-05,3.717e-05,8.380e-02,6.964e-05,2.858e-06,4.045e-03,1.085e-04,2.621e-04,3.319e-06,8.434e-04,4.455e-03,2.501e-04,6.579e-06,0.423,9.160e-06,1.159e-05,1.661e-03,0.020,0.053,6.654e-05,9.059e-03,3.507e-05,6.505e-04,5.178e-04,2.854e-05,1.915e-04,8.393e-06,1.625e-01,3.908e-03,0.059,0.013,0.043,1.313e-04
1,874263,9.663e-05,0.005,9.355e-07,3.751e-05,4.077e-06,2.389e-07,1.132e-05,2.261e-02,1.984e-04,2.259e-05,3.194e-06,3.086e-06,3.190e-04,2.054e-05,9.797e-07,0.277,3.194e-06,2.808e-06,1.351e-04,0.035,0.575,8.573e-06,2.107e-03,1.433e-04,9.558e-05,3.780e-05,1.001e-05,3.087e-06,1.985e-06,1.294e-04,8.325e-06,0.008,0.061,0.014,8.000e-05
2,874264,6.736e-05,0.198,6.141e-06,2.124e-04,7.005e-02,6.549e-06,2.514e-06,1.432e-02,6.515e-04,2.534e-04,1.611e-04,7.956e-03,1.307e-04,5.707e-05,1.641e-05,0.019,7.576e-05,2.707e-06,9.518e-03,0.023,0.145,1.215e-06,1.256e-03,7.252e-04,3.161e-04,3.606e-05,3.877e-05,1.102e-04,5.919e-05,3.665e-03,2.458e-04,0.029,0.357,0.118,1.979e-04
3,874265,6.967e-04,0.029,3.461e-06,1.309e-05,1.864e-02,1.241e-05,2.305e-06,1.213e-02,6.565e-03,1.591e-05,1.097e-05,1.423e-03,9.784e-05,1.969e-05,3.781e-06,0.122,8.539e-06,1.352e-06,1.326e-04,0.006,0.700,3.483e-05,8.458e-05,3.220e-04,2.706e-05,5.642e-04,9.595e-06,2.767e-06,2.247e-06,5.337e-02,3.054e-06,0.017,0.003,0.028,1.382e-04
4,874266,2.122e-04,0.151,4.230e-06,6.719e-05,2.408e-02,8.666e-06,1.868e-06,5.115e-04,8.937e-04,3.886e-05,1.506e-05,3.084e-03,5.830e-04,3.931e-05,3.847e-06,0.105,6.509e-04,1.933e-06,4.642e-02,0.130,0.334,1.852e-06,1.590e-03,4.888e-05,5.988e-05,1.896e-03,1.915e-05,9.977e-05,4.747e-05,3.280e-03,7.794e-05,0.065,0.117,0.013,2.555e-04


In [262]:
predictions.to_csv('may27testsubmission2.csv',index=False)